# Gruppe 1

## Présentation


# 1- Collecting and cleanig of the Data

In [142]:
from pycoshark.utils import create_mongodb_uri_string
from bson import ObjectId
import pymongo
import pandas as pd
import datetime

# Database credentials
user = 'grundlagenki2021'
password = '38BTGx71d'
host = '134.76.81.151'
port = '27017'
authentication_db = 'smartshark'
database = "smartshark"
ssl_enabled = None

# Establish connection
uri = create_mongodb_uri_string(user, password, host, port, authentication_db, ssl_enabled)

myclient = pymongo.MongoClient(uri)
mydb = myclient["smartshark"]

# Calcul of the time needed for an Issue to be done
""" We assume that an issue start with an Event status "open" and is solved only when the event status is
 "closed".
 """
def time_needed(iId):
    mycol = mydb["event"]
    #issue when the value is closed
    oi = ObjectId(iId)

    mydoc1 = mycol.find({'status':"status",'new_value':"Closed",'issue_id':oi})
    df1 =  pd.DataFrame(mydoc1)
    time_close = df1.loc[0]["created_at"]


    #issue when the value is open
    mydoc2 = mycol.find({'status':"status",'new_value':"Open",'issue_id':oi})
    df2 =  pd.DataFrame(mydoc2)
    time_open = df2['created_at'].loc[df2.index[0]]

    time = time_close - time_open
    time_hours = time.total_seconds()/3600
    return time_hours



In [143]:
time_needed(iId='58bfd11a0ccb2667fe27318b')

25269105.855

## 1.1- Event

    1.1.1 Get the data from the Database and removing unnecessary field.

In [ ]:
lmt = 8000
mycol = mydb["event"]
#Dataframe when the value is closed
mydoc1 = mycol.find({'status':"status",'new_value':"Closed"}).limit(lmt)
df1 =  pd.DataFrame(list(mydoc1))
df1.drop(['external_id','author_id','status','_id','old_value'], axis=1,inplace=True)
#Dataframe when the value is open
mydoc2 = mycol.find({'status':"status",'new_value':"Open"}).limit(lmt)
df2 =  pd.DataFrame(list(mydoc2))
df2.drop(['external_id','author_id','status','_id','old_value'], axis=1,inplace=True)
#merging the value
df3 = pd.merge(df1,df2,how='inner',on='issue_id')

df3.to_csv('event.csv', encoding='utf-8', index=False)



    1.1.2 Join the dataframe with the column of the time necessary to be solved

## 1.2- Issue

    1.2.1 Get the data from the Database and removing unnecessary field.

In [ ]:
mycol = mydb["issue"]
mydoc = mycol.find().limit(5000)
df =  pd.DataFrame(mydoc)
df.drop(['external_id','issue_system_id','title','desc','created_at','updated_at','creator_id','reporter_id','affects_versions','resolution','fix_versions','issue_links','assignee_id','parent_issue_id','original_time_estimate','labels','status','environment','issue_type_manual','issue_type_verified'], axis=1,inplace=True)
df.dropna(inplace=True)
df.to_csv('issue.csv', encoding='utf-8', index=False)


    1.2.2 Selection of issues that have been solved

In [ ]:
df_csv = pd.read_csv('issue.csv')
mycol = mydb["event"]

#df[df[mycol.find({'status':"status",'new_value':"Open",'issue_id':col_obj.values}).count()>0]]
list = []
for i in range (314):
    oid = df_csv.iloc[i]['_id']
    print(i)
    if mycol.find({'status':"status",'new_value':"Open",'issue_id':ObjectId(f'{oid}')}).count()==0 or mycol.find({'status':"status",'new_value':"Closed",'issue_id':ObjectId(f'{oid}')}).count()==0:

        list.append(i)

df_csv.drop(list,inplace=True)
df_csv.to_csv('issue.csv', encoding='utf-8', index=False)


    1.2.3 Join the dataframe with the column of the time necessary to be solved

In [144]:
list = []

for i in range (314):
    list.append(time_needed(df_csv.iloc[i]['_id']))

df_wtn = df_csv.assign(time_needed = list)
df_wtn.to_csv('issue.csv', encoding='utf-8', index=False)


    1.2.4 issue type with the time necessary to be done

In [136]:
df_it = pd.read_csv('issue.csv')
df_it.drop(['priority','components'],inplace=True,axis=1)
d = df_it.groupby('issue_type')
df_it.to_csv('issue@issue_type.csv', encoding='utf-8', index=False)

,_id,time_needed
issue_type,,
Bug,58bfd11a0ccb2667fe27318b,292 days 11:11:45.855000
Improvement,58bfcb820ccb2667fe26af47,28 days 20:47:58.040000
New Feature,58bfcb8d0ccb2667fe26b065,74 days 23:55:25.285000
Sub-task,58bfd1cc0ccb2667fe273e6b,280 days 23:43:54.149000
Task,58bfcbf00ccb2667fe26b91c,191 days 22:54:38.682000


    1.2.5 priority with the time necessary to be done

In [118]:
dp = pd.read_csv('issue.csv')
dp.drop(['issue_type','components'],inplace=True,axis=1)
dp.to_csv('issue@priority.csv', encoding='utf-8', index=False)